In [21]:
import numpy as np
import pickle
import torch 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from fastprogress import progress_bar

In [2]:
with open('data/board_states.pkl', 'rb') as f:
    x = pickle.load(f)
with open('data/results.pkl', 'rb') as f:
    y = pickle.load(f)

In [32]:
class ChessDataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = torch.from_numpy(np.asarray(self.x))
        y = torch.from_numpy(np.asarray(self.y))
        
        x = x.type(torch.FloatTensor)
        y = y.type(torch.FloatTensor)
        
        sample = {'x': x, 'y': y}

        return sample
    
dataset = ChessDataset(x, y)

In [33]:
dataloader = DataLoader(
    dataset, 
    batch_size=4,
    shuffle=True, 
    num_workers=4
)

In [34]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(8*8*12, 1)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
        return x
    
model = Model()

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [36]:
for epoch in range(2):
    running_loss = 0.0
    for data in progress_bar(dataloader):
        inputs, results = data['x'], data['y']

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, results)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 0: 
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/10))
            running_loss = 0.0

print('Finished Training')

RuntimeError: size mismatch, m1: [94237696 x 12], m2: [768 x 1] at ../aten/src/TH/generic/THTensorMath.cpp:197